<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_A_FLUX_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [7]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_A-FLUX',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/510Normal.png', '/content/Gold

In [10]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_A-FLUX',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_A-FLUX/510Normal.png', '/content/Gold

In [11]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 736ms/step - kl_loss: 242.4468 - loss: 44127.5000 - reconstruction_loss: 43885.0508
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 83ms/step - kl_loss: 1002.3146 - loss: 25606.8105 - reconstruction_loss: 24604.4980
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - kl_loss: 198.1039 - loss: 22801.5117 - reconstruction_loss: 22603.4082
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - kl_loss: 127.0947 - loss: 21978.1777 - reconstruction_loss: 21851.0820
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - kl_loss: 131.6042 - loss: 20616.6523 - reconstruction_loss: 20485.0488
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - kl_loss: 121.8399 - loss: 18776.9062 - reconstruction_loss: 18655.0664
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - kl_loss: 103.5901 - loss: 17714.6445 - reconstruction_loss: 17611.0547
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - kl_loss: 95.6290 - loss: 17284.2324 - reconstruction_loss: 17188.6035
Epoch

In [12]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_A-FLUX',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [13]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=6a4e69b1b4fd86fad9c5c6f7b22a701d30bf9b4a7a604ccd16e47e34bc119576
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [14]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
0.0030187775684221547
48.285282
[[-5.6133776  5.619455  -5.000528 ]]
[[-3.8584192 -3.9091134 -3.6421125]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
0.003257929885258111
53.39478
[[-4.842755   6.8854947 -5.222973 ]]
[[-3.1032763 -4.706269  -3.761054 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
0.003878257471163167
57.982166
[[-2.4779365  9.583222  -3.2993305]]
[[-1.9093086 -5.1291537 -2.8500235]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
0.00296288830082634
53.098682
[[-5.7138786  6.0395017 -5.315647 ]]
[[-3.6745512 -4.327196  -3.7535172]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
0.0034393817641880098
50.261787
[[-5.46135    5.9640756 -5.1498747]]
[[-3.6810746 -4.14172   -3.7176442]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.0029170802286702328
45.86801
[[-4.447028   6.5055985 -4.6554866]]
[[-3.2903907 -4.0960617 -3.4928534]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
0.0030172339792454723
46.98567
[[-5.520677   5.557901  -4.9236717]]
[[-3.8856165 -3.8002484 -3.6046987]]
1

In [15]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('A-FLUX_train.csv')

In [16]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.003665431075071803
47.68122
[[-2.16772    8.678855  -2.9646966]]
[[-2.1824908 -4.400956  -2.7810347]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.003705802819327367
48.04696
[[-2.27539    8.656687  -3.0614579]]
[[-2.2402027 -4.4099374 -2.774671 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.00355482814059464
51.01904
[[-2.3698514  8.885537  -3.2623076]]
[[-2.130072  -4.668291  -2.8415396]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.0035321667943718354
55.085205
[[-2.567118   9.203334  -3.4411025]]
[[-2.0546045 -4.9614625 -2.82739  ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
0.003583566121072322
48.646297
[[-2.40926    8.6417265 -3.1777058]]
[[-2.2717679 -4.4609447 -2.8027813]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.0035718475534084245
48.67774
[[-2.3929298  8.657853  -3.1587718]]
[[-2.2671943 -4.458338  -2.7910318]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.003582261239323696
49.104084
[[-2.21279   8.800973 -3.0358  ]]
[[-2.1580253 -4.4975576 -2.795137 ]]
1/1 ━

In [17]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('A-FLUX_testnorm.csv')

In [18]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
0.05497399664206166
32.38998
[[ 0.18585774  7.4675345  -1.5625278 ]]
[[-2.554914  -2.8238294 -4.0058184]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.008157460409958227
36.552544
[[-2.9674914  6.8347516 -3.3242211]]
[[-3.1138098 -3.3726861 -2.9153981]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.01899355317684077
36.3526
[[-2.1378891  7.259161  -3.0299513]]
[[-2.7844703 -3.360614  -2.9655097]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.0030045720855222044
48.080906
[[-5.4112773  5.853111  -4.92416  ]]
[[-3.7298698 -3.9871757 -3.5863419]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.020029109400143918
32.17477
[[-1.9768935  6.819386  -2.8146892]]
[[-2.943566  -2.9648955 -2.9498818]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.020468858058780147
31.518446
[[-2.1149452  6.684686  -2.8243783]]
[[-2.8975081 -2.9760194 -2.8650963]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.0029820600097064786
56.221638
[[-5.7577405  6.3508353 -5.4739213]]
[[-3.5073953 -4.6334586 -3.7916677]]


In [19]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('A-FLUX_testanom.csv')